In [1]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.3.0


In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import os
plt.style.use('ggplot')
%matplotlib inline 

In [3]:
img_width = 64
img_height = 64 